In [1]:
import math
import cv2
import numpy as np
from time import time
from threading import Timer
import mediapipe as mp
import matplotlib.pyplot as plt
from IPython.display import HTML
from pygame import mixer
import tempfile
from gtts import gTTS

pygame 2.1.3 (SDL 2.0.22, Python 3.8.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mp_pose = mp.solutions.pose
pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils   

In [3]:
# def mediapipe_detection(image, model):
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
#     image.flags.writeable = False                  # Image is no longer writeable
#     results = model.process(image)                 # Make prediction
#     image.flags.writeable = True                   # Image is now writeable 
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
#     return image, results

In [4]:
# cap_detect(pose)

In [5]:
# 計算角度的副函式
def calculateAngle(landmark1, landmark2, landmark3):
    # 獲取所需座標
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
    # 計算三點之間的夾角
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    return angle

In [6]:
def speak(sentence, lang):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts=gTTS(text=sentence, lang=lang)
        tts.save('{}.mp3'.format(fp.name))
        mixer.init()
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play(1)

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    height, width, _ = image.shape
    landmarks = []
    if results.pose_landmarks:
            # 畫關鍵點在圖片上
        mp_drawing.draw_landmarks(image=image, landmark_list=results.pose_landmarks,
                    connections=mp_pose.POSE_CONNECTIONS)
        for landmark in results.pose_landmarks.landmark:
        # 將關鍵點加進list內.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                    (landmark.z * width)))    
    try:
        l_elbow = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
        if l_elbow>180:
            l_elbow=360-l_elbow
    except:
        l_elbow=0
    try:
        # 取得右肩、右肘和右腕之間的夾角。    # 12 14 16 
        r_elbow = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
        if r_elbow>180:
            r_elbow=360-r_elbow
    except:
        r_elbow=0
    try:
        # 取得左肘、左肩和左臀之間的夾角。    # 13 11 23
        l_shoulder = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
        if l_shoulder>180:
            l_shoulder=360-l_shoulder
    except:
        l_shoulder=0
    try:
        # 取得右臀、右肩和右肘之間的夾角。    # 14 12 24
        r_shoulder = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
        if r_shoulder>180:
            r_shoulder=360-r_shoulder
    except:
        r_shoulder=0
    try: # 左臀角度 
        l_hip = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])
        if l_hip>180:
            l_hip=360-l_hip            
    except:
        l_hip=0
    try: # 右臀角度
        r_hip = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
        if r_hip>180:
            r_hip=360-r_hip            
    except:
        r_hip=0
    try:
        # 取得左臀、左膝蓋和左腳踝之間的角度。# 23 25 27
        l_knee = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
        if l_knee>180:
            l_knee=360-l_knee
    except:
        l_knee=0        
    try:
        # 取得右臀、右膝蓋和右腳踝之間的角度。# 24 26 28 
        r_knee = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
        if r_knee>180:
            r_knee=360-r_knee
    except:
        r_knee=0
    
    return image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee

In [17]:
def ep4_baby1():
    lastTime1 = time()
    lastTime2 = time()
    lastTime3 = time()
    lastTime4 = time()
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(static_image_mode=False, 
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5, 
                      model_complexity=2)as pose:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            # Make detection
            output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)
          
            condA = l_knee>50 or r_knee>50
            condB = l_shoulder<150 or r_shoulder<150    
            condC = l_hip>70 or r_hip>70
            # and (time()-lastTime)>5
            if condA and (time()-lastTime1)<3:
                text1= 'hip back on heel'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                # lastTime = time()
            elif condA and (time()-lastTime1)>3 :
                text1= 'hip back on heel'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                lastTime1 = time()
                speak("臀往後坐在腳跟上",'zh-tw')
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<3:
                    text2= 'hands straight forward'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>3:
                    speak("雙手伸直往前延伸",'zh-tw')
                    text2= 'hands straight forward'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    lastTime2 = time()
                else:
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condC and (time()-lastTime3)<3:
                        text3= 'hip back on heel 2'
                        cv2.putText(output_image, text3, (30, 90),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)    
                    elif condC and (time()-lastTime3)>3:
                        speak("臀往後坐在腳跟上",'zh-tw')
                        text3= 'hip back on heel 2'
                        cv2.putText(output_image, text3, (30, 90),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        lastTime3 = time()
                    elif (time()-lastTime4)>3:
                        text3= ' success '
                        speak("完美!",'zh-tw')
                        cv2.putText(output_image, text3, (30, 90),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        lastTime4 = time()
                        break


            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()


In [18]:
ep4_baby1()

In [10]:
def test_hand():
    lastTime1 = time()
    lastTime2 = time()
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(static_image_mode=False, 
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5, 
                      model_complexity=2)as pose:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA = l_elbow<140 
            condB = r_elbow<140   
             
            if condA and (time()-lastTime1)<3:
                text1= 'left hand stright'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>3 :
                text1= 'left hand stright'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                lastTime1 = time()
                speak("左手打平",'zh-tw')
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<3 :
                    text2= 'right hand straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>3 :
                    speak("右手打平",'zh-tw')
                    text2= 'right hand straight'
                    lastTime2 = time()
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                else:
                    text2= ' success '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
        

            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()


In [11]:
test_hand()